The following workbook is to scrape data from the url: <https://ca.milesplit.com/meets/493916-cvaa-preview-2022/results/846055/raw>

As our very first step, let's extract the race_id from the url programatically. This will be saved in our output dataframe and as part of the name of our output file.

In the space below, generate a variable called race_id and assign it the six digit value before the race name in the URL.  For example, in the url <https://ca.milesplit.com/meets/493916-cvaa-preview-2022/results/846055/raw> the race_id should be 123456.  Verify that your code works by extracting the race_id from both the example url and your primary url. 

In [14]:
import re

url = 'https://ca.milesplit.com/meets/493916-cvaa-preview-2022/results/846055/raw'

def extract_race_id(url):
    #define your function here!
    return re.search(r'/meets/(\d{6})', url).group(1)

race_id = extract_race_id(url)

Now lets process the HTML file! 

To get you started I've saved a file to the raw_html_files folder with example webpage (html) code. Change the file path to match the file path on your computer. Verify that the html file is being read correctly.

In [15]:
html_file_path = r"C:\Users\abw13\Research\research_assistants copy\research_assistants copy\raw_html_files\meet_493916.html"

In [16]:
import pandas as pd
from bs4 import BeautifulSoup
import os

with open(html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')
soup.prettify()[:500]  # This will display the HTML content in a structured format

'<!DOCTYPE html>\n<html lang="en" xmlns:="">\n <head>\n  <script src="https://cmp.osano.com/AzyWAQS5NWEEWkU9/eab0a836-8bac-45b1-8b3e-e92e57e669db/osano.js?language=en">\n  </script>\n  <script src="https://www.flolive.tv/osano-flo.js">\n  </script>\n  <!-- Google Tag Manager -->\n  <script>\n   (function (w, d, s, l, i) {\n            w[l] = w[l] || [];\n            w[l].push({\n                \'gtm.start\':\n                    new Date().getTime(), event: \'gtm.js\'\n            });\n            var f = d.getEle'

If the html displayed above you have read in your file!

Next we need to identify the correct portion of the html file with the individual results table that we want to scrape and format. Do so below.

In [17]:
def find_results_table(html):
    """
    Return the first <table>...</table> found inside
    <div id="meetResultsBody"> as an HTML string.
    """
    # 1) Find the meetResultsBody div
    div_pat = re.compile(
        r'<div[^>]*\bid=(["\'])meetResultsBody\1[^>]*>(.*?)</div>',
        re.IGNORECASE | re.DOTALL
    )
    m = div_pat.search(html)
    if not m:
        return None

    # 2) From its inner HTML, pull out the first <table>...</table>
    inner = m.group(2)
    table_pat = re.compile(
        r'<table\b[^>]*>.*?</table>',
        re.IGNORECASE | re.DOTALL
    )
    t = table_pat.search(inner)
    return t.group(0) if t else None

tablestring = find_results_table(str(soup))

Next, transform the content in your html table or text into a pandas dataframe.  The pandas dataframe output must have the following column names:
- race_id
- race_url
- race_name
- place
- athlete
- athlete_url
- grade
- team
- team_url
- finish
- point  

Get race_id from your generated variable above.  Get the race_url from the provided url.

If your text or table do not have the appropriate column names rename the columns or create the columns even if they are empty.

In [18]:
import re
import pandas as pd
from bs4 import BeautifulSoup

def _extract_race_id_from_text(s: str):
    m = re.search(r'/results/(\d+)/', s or "")
    return m.group(1) if m else None

def generate_dataframe(table, raceid, url):
    """
    Convert the HTML produced by find_results_table(...) into a pandas DataFrame.

    Expected output columns:
      race_id, race_url, race_name, place, athlete, athlete_url,
      grade, team, team_url, finish, point
    """
    # Accept either a BeautifulSoup Tag or a raw HTML string
    soup = BeautifulSoup(str(table), "html.parser")

    data_rows = []
    race_name = None

    for tr in soup.find_all("tr"):
        tds = tr.find_all("td")
        if not tds:
            continue

        # Single-cell row = race header (e.g., "Girls Sophomore - 3 Mile Run")
        if len(tds) == 1:
            header = tds[0].get_text(strip=True)
            if header:                 # ignore empty separators like <tr><td></td></tr>
                race_name = header
            continue

        # Skip column header row: Place | Athlete | Grade | School | Time
        if tds[0].get_text(strip=True).lower() == "place":
            continue

        # Normal result rows (need at least 5 cells)
        if len(tds) >= 5 and race_name:
            place   = tds[0].get_text(strip=True)
            athlete = tds[1].get_text(strip=True)
            grade   = tds[2].get_text(strip=True)
            team    = tds[3].get_text(strip=True)
            finish  = tds[4].get_text(strip=True)

            # Optional hyperlinks for athlete/team
            a_tag = tds[1].find("a")
            t_tag = tds[3].find("a")
            athlete_url = a_tag["href"].strip() if a_tag and a_tag.has_attr("href") else None
            team_url    = t_tag["href"].strip() if t_tag and t_tag.has_attr("href") else None

            data_rows.append({
                "race_id": raceid,
                "race_url": url,
                "race_name": race_name,
                "place": place,
                "video": None,
                "athlete": athlete,
                "athlete_url": athlete_url,
                "grade": grade,
                "team": team,
                "team_url": team_url,
                "finish": finish,
                "point": None,   # not available in the HTML; keep placeholder
            })

    df = pd.DataFrame(
        data_rows,
        columns=[
            "race_id","race_url","race_name","place", "video", "athlete","athlete_url",
            "grade","team","team_url","finish","point"
        ],
    )

    # Optional: numeric cleanup
    if not df.empty:
        df["place"] = pd.to_numeric(df["place"], errors="coerce")
        df["grade"] = pd.to_numeric(df["grade"], errors="coerce")

    return df

df = generate_dataframe(tablestring, race_id, url)
df


,race_id,race_url,race_name,place,video,athlete,athlete_url,grade,team,team_url,finish,point
0,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Girls Freshmen - 3 Mile Run,1,None,Kayla Skinner,None,9,Trabuco Hills (SS),None,20:42.9,None
1,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Girls Freshmen - 3 Mile Run,2,None,Janessa Harris,None,9,Dana Hills (SS),None,21:22.3,None
2,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Girls Freshmen - 3 Mile Run,3,None,Ariana Price,None,9,San Juan Hills (SS),None,22:05.8,None
3,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Girls Freshmen - 3 Mile Run,4,None,Addison Hansen,None,9,San Juan Hills (SS),None,22:08.5,None
4,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Girls Freshmen - 3 Mile Run,5,None,Maya Aoude,None,9,Aliso Niguel (SS),None,22:14.5,None
...,...,...,...,...,...,...,...,...,...,...,...,...
419,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Boys JV - 3 Mile Run,69,None,Sanjeet Veeravalli,None,11,El Toro (SS),None,21:50.1,None
420,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Boys JV - 3 Mile Run,70,None,Bryan Reynoso,None,11,Mission Viejo (SS),None,22:03.7,None
421,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Boys JV - 3 Mile Run,71,None,Tyler Blick,None,11,Mission Viejo (SS),None,22:33.9,None
422,493916,https://ca.milesplit.com/meets/493916-cvaa-pre...,Boys JV - 3 Mile Run,72,None,Zen Pilapil,None,11,Tesoro (SS),None,22:59.5,None


In [19]:
expected_columns = [
    'race_id', 'race_url', 'race_name','place', 'video', 'athlete', 'athlete_url',
    'grade', 'team', 'team_url', 'finish', 'point'
]

def verify_dataframe_columns(df, expected_columns):
    return list(df.columns) == expected_columns

def verify_dataframe_is_not_empty(df):
    return not df.empty

# Example usage:
df_columns_correct = verify_dataframe_columns(df, expected_columns)
df_is_not_empty = verify_dataframe_is_not_empty(df)

print("Columns are correct:", df_columns_correct)
print("DataFrame is not empty:", df_is_not_empty)

Columns are correct: True
DataFrame is not empty: True


Take time to verify the following about your dataframe: 
1. The dataframe is not empty.
2. Column names are correct and in the correct order
3. The data in the dataframe matches the data at the website url.

If anything is incorrect in your dataframe - iterate in the space above until it is correct!

Now generate the correct file name for your dataframe using the url your only input.  The URL for the HTML file is: <https://ca.milesplit.com/meets/493916-cvaa-preview-2022/results/846055/raw>

Below generate a file name in the format TABLETYPE_results_meet_MEETID.csv.  For example, individual_results_meet_123456.csv.  Options for TableType are individual or team for individual or team results.

In [20]:
def generate_filename(url, table_type):
    # Your code to generate the filename based on the URL and table type
    url = extract_race_id(url)
    return f'{table_type}_results_meet_{url}'


Finally, generate the correct file path, so that this csv saves in the folder 'output' in the 'research_assistant' folder.

In [21]:
filename = generate_filename(url, "individual")

In [22]:
if df_columns_correct and df_is_not_empty:
    print("DataFrame columns match the expected columns.")
    output_file_locatin = os.path.join(file_path, filename)
    df.to_csv(filename, index=False)
else:
    print("DataFrame columns do not match the expected columns or DataFrame is empty.")

DataFrame columns match the expected columns.
